In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import itertools as it
from MetricMethod import *
from phik import phik_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [ ]:
df = pd.read_csv("Bank Customer Churn Prediction.csv")
df.info()

In [ ]:
df['credit_score'].describe()

In [ ]:
for coulumn in it.filterfalse(lambda x: x in 
                            set(['customer_id', 'country', 'gender', 'churn', 'products_number']),
                            it.islice(df.columns, 0, None)):
    mean = df[coulumn].mean()
    std = df[coulumn].std()

    lower_limit = mean - 3 * std
    upper_limit = mean + 3 * std

    ind = (df[coulumn] < lower_limit) | (df[coulumn] > upper_limit)
    out = df[ind][['customer_id', coulumn]]
    if len(out) > 0:
        print(lower_limit, upper_limit)
        print(out)


In [ ]:
df['gender'].value_counts().plot.bar(grid=True, figsize=(4, 4))

In [ ]:
df['country'].value_counts().plot.bar(grid=True, figsize=(4, 4))

In [ ]:
df.groupby('gender')['churn'].count().plot.bar(ylabel='count')

In [ ]:
df['churn'].value_counts().plot.bar(grid=True).set_title("churn")
plt.ylabel("count")
plt.show()

In [ ]:
t = df.drop(columns=['customer_id', ])
t.hist(figsize=(8, 12), xrot=15)


In [ ]:
t = df.drop(columns=['customer_id', 'active_member', 'country', 'gender',
                      'churn', 'products_number', 'credit_card'])

# Получение количества колонок
num_columns = len(t.columns)

# Расчет количества строк и столбцов для сетки
num_rows = int((num_columns + 1) / 2)  # Округляем вверх до ближайшего целого числа

# Создание сетки подграфиков
fig, axes = plt.subplots(num_rows, 2, figsize=(8, 10))

# Распаковка двумерного массива осей в одномерный массив
axes = axes.flatten()

# Построение ящиковых диаграмм для каждой колонки
for i, column in enumerate(t.columns):
    # Построение ящиковой диаграммы для текущей колонки
    axes[i].boxplot(t[column].astype(float))
    axes[i].grid(True)
    
    # Установка заголовка для текущего графика
    axes[i].set_title(column)

# Удаление пустых подграфиков
if num_columns % 2 != 0:
    fig.delaxes(axes[-1])

# Расположение подграфиков
fig.tight_layout()

# Отображение графиков
plt.show()

In [ ]:
t = df[['active_member', 'country', 'gender',
                      'churn', 'products_number', 'credit_card']]

# Получение количества колонок
num_columns = len(t.columns)

# Расчет количества строк и столбцов для сетки
num_rows = int((num_columns + 1) / 2)  # Округляем вверх до ближайшего целого числа

# Создание сетки подграфиков
fig, axes = plt.subplots(num_rows, 2, figsize=(8, 10))

# Распаковка двумерного массива осей в одномерный массив
axes = axes.flatten()

# Построение ящиковых диаграмм для каждой колонки
for i, column in enumerate(t.columns):
    # Построение ящиковой диаграммы для текущей колонки
    #axes[i].plot(t[column], kind='bar')
    t[column].value_counts().plot(kind='bar', ax=axes[i])
    axes[i].grid(True)
    
    # Установка заголовка для текущего графика
    axes[i].set_title(column)

# Удаление пустых подграфиков
if num_columns % 2 != 0:
    fig.delaxes(axes[-1])

# Расположение подграфиков
fig.tight_layout()

# Отображение графиков
plt.show()

In [ ]:
df.corr().round(4)

In [ ]:
phik_matrix(df)

In [ ]:
df.groupby('country')['estimated_salary'].agg(['mean', 'median']).plot.bar(ylabel='estimated_salary', grid = True)

In [ ]:
df.boxplot(column='estimated_salary', by='country', grid=True)

    


In [ ]:
t = df.copy()
t['group_age'] = pd.qcut(t['age'], 5)
t.boxplot(column='balance', by='group_age')

In [ ]:
t.hist(column='balance', by='group_age', figsize=(8, 15))

In [ ]:
t.hist(column='products_number', by='group_age', figsize=(8, 13))

# Настройка внешнего вида графика и добавление названия
plt.suptitle('Гистограмма типа продукта по возрастным группам')

# Показываем график
plt.show()

In [ ]:
t.hist(column='products_number', by='churn')

# Настройка внешнего вида графика и добавление названия
plt.suptitle('products_number group by churn')

# Показываем график
plt.show()

In [ ]:
df.groupby('credit_card')['credit_card'].count().plot.bar(grid=True, ylabel='count')

In [ ]:
t.hist(column='active_member', by='churn')
# Настройка внешнего вида графика и добавление названия
plt.suptitle('active_member group by churn')

# Показываем график
plt.show()

In [ ]:
t.hist(column='country', by='churn')
# Настройка внешнего вида графика и добавление названия
plt.suptitle('country group by churn')

# Показываем график
plt.show()

In [ ]:
t.boxplot(column='age', by='group_age', figsize=(8, 10))

plt.suptitle('age group by group_age')

# Показываем график
plt.show()

In [ ]:
df_processed = pd.get_dummies(df, columns=['country', 'gender'], drop_first=True)
df_processed = df_processed.drop(columns='customer_id')
df_processed

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()

names = df_processed.columns
d = scaler.fit_transform(df_processed)

df_processed = pd.DataFrame(d, columns=names)
df_processed.head()

In [ ]:
# Разделите датафрейм на тренировочный и остаточный набор данных
train_df, temp_df = train_test_split(df_processed, test_size=0.3, random_state=42)

train_X = train_df.drop(columns='churn')
train_y = train_df['churn']

# Разделите остаточный набор данных на валидационный и тестовый наборы данных
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

val_X = val_df.drop(columns='churn')
val_y = val_df['churn']

test_X = test_df.drop(columns='churn')
test_y = test_df['churn']

# Выведите размеры полученных наборов данных
print("Размер тренировочного набора данных:", train_df.shape)
print("Размер валидационного набора данных:", val_df.shape)
print("Размер тестового набора данных:", test_df.shape)

In [ ]:
train_X.head()

In [ ]:
k = 3
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(train_X, train_y)
pred_y = model.predict(test_X)

print(f'Accuracy {accuracy_score(test_y, pred_y)}')
print(f'F1 {f1_score(test_y, pred_y)}')
print(f'Precision {precision_score(test_y, pred_y)}')
print(f'Recall {recall_score(test_y, pred_y)}')

In [ ]:
k = 1
model = KNeighborsClassifier(n_neighbors=k)
model.fit(train_X, train_y)
pred_y = model.predict(test_X)

print(f'Accuracy {accuracy_score(test_y, pred_y)}')
print(f'F1 {f1_score(test_y, pred_y)}')
print(f'Precision {precision_score(test_y, pred_y)}')
print(f'Recall {recall_score(test_y, pred_y)}')

In [ ]:
smote = SMOTE()
train_X_sm, train_y_sm = smote.fit_resample(train_X, train_y)

In [ ]:
k = 1
model = KNeighborsClassifier(n_neighbors=k)
model.fit(train_X_sm, train_y_sm)
pred_y = model.predict(test_X)

print(f'Accuracy {round(accuracy_score(test_y, pred_y), 4)}')
print(f'F1 {round(f1_score(test_y, pred_y), 4)}')
print(f'Precision {round(precision_score(test_y, pred_y), 4)}')
print(f'Recall {round(recall_score(test_y, pred_y), 4)}')

In [ ]:
model = KNN(countNeigbor=1)
model.fit(train_X_sm, train_y_sm)
pred_y = model.predict(test_X)

print(f'Accuracy {round(accuracy_score(test_y, pred_y), 4)}')
print(f'F1 {round(f1_score(test_y, pred_y), 4)}')
print(f'Precision {round(precision_score(test_y, pred_y), 4)}')
print(f'Recall {round(recall_score(test_y, pred_y), 4)}')

In [ ]:
model = KNN_weight(countNeigbor=1)
model.fit(train_X_sm, train_y_sm)
pred_y = model.predict(test_X)

print(f'Accuracy {round(accuracy_score(test_y, pred_y), 4)}')
print(f'F1 {round(f1_score(test_y, pred_y), 4)}')
print(f'Precision {round(precision_score(test_y, pred_y), 4)}')
print(f'Recall {round(recall_score(test_y, pred_y), 4)}')

In [ ]:
model = ParzenWindowVariableWidth(countNeighbor=3)
model.fit(train_X_sm, train_y_sm)
pred_y = model.predict(test_X)

print(f'Accuracy {round(accuracy_score(test_y, pred_y), 4)}')
print(f'F1 {round(f1_score(test_y, pred_y), 4)}')
print(f'Precision {round(precision_score(test_y, pred_y), 4)}')
print(f'Recall {round(recall_score(test_y, pred_y), 4)}')

In [ ]:
model = ParzenWindowFixedWidth(width=0.4)
model.fit(train_X_sm, train_y_sm)
pred_y = model.predict(test_X)

print(f'Accuracy {round(accuracy_score(test_y, pred_y), 4)}')
print(f'F1 {round(f1_score(test_y, pred_y), 4)}')
print(f'Precision {round(precision_score(test_y, pred_y), 4)}')
print(f'Recall {round(recall_score(test_y, pred_y), 4)}')

In [ ]:
model = PotentialFunction(width=0.4, eps=0.5, method='kdtree')
model.fit(train_X_sm, train_y_sm)
pred_y = model.predict(test_X)

print(f'Accuracy {round(accuracy_score(test_y, pred_y), 4)}')
print(f'F1 {round(f1_score(test_y, pred_y), 4)}')
print(f'Precision {round(precision_score(test_y, pred_y), 4)}')
print(f'Recall {round(recall_score(test_y, pred_y), 4)}')

In [ ]:
y_pred = np.array([0 for i in range(len(test_y))])
accuracy_score(test_y, pred_y)